In [1]:
import glob
import tifffile
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
from mpl_toolkits.mplot3d import axes3d, Axes3D
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
from numpy import *
#from sklearn.cluster import OPTICS, cluster_optics_dbscan 
from sklearn.preprocessing import normalize, StandardScaler 
from os import listdir
from os.path import isfile, join
from sklearn.manifold import TSNE

import re
%matplotlib notebook

# Path of Data

In [2]:
in_out_germinal = r"Y:\coskun-lab\Shambavi\Paper\Data\Main Data\New Data\Border Dataset\Border Dataset"
in_germinal = r"Y:\coskun-lab\Shambavi\Paper\Data\Main Data\New Data\In Germinal Dataset\In Germinal Dataset"
out_germinal = r"Y:\coskun-lab\Shambavi\Paper\Data\Main Data\New Data\Outside Dataset\Outside Dataset"
unlabelled_tissue = r"Y:\coskun-lab\Shambavi\Paper\Data\Main Data\New Data\Unlabeled Dataset\Unlabeled Dataset"
in_out_skew = "Y:\coskun-lab\Shambavi\Paper\Data\Old Extra Data\Data-Hyperion-Exp1"


# Obtaining Files

In [29]:
#in_out_germinal_files = [f for f in listdir(in_out_germinal) if isfile(join(in_out_germinal, f))]
#out_germinal_files = [f for f in listdir(out_germinal) if isfile(join(out_germinal, f))]
# in_germinal_files = [f for f in listdir(in_germinal) if isfile(join(in_germinal, f))]
unlabelled_tissue_files = [f for f in listdir(unlabelled_tissue) if isfile(join(unlabelled_tissue, f))]


# Saving labels 

In [30]:
def getlabels(onlyfiles):
    labels = []
    for i in range(len(onlyfiles)):
        labels.append(onlyfiles[i].split()[-1][:-4])
    return labels

In [31]:
# in_out_germinal_labels = getlabels(in_out_germinal_files) 
#out_germinal_labels = getlabels(out_germinal_files)
# in_germinal_labels = getlabels(in_germinal_files)
unlabelled_tissue_labels = getlabels(unlabelled_tissue_files)
# in_out_skew_labels = getlabels(in_out_skew_files)

In [32]:
# np.save('in_out_germinal_labels',in_out_germinal_labels)
#np.save('out_germinal_labels',out_germinal_labels)
# np.save('in_germinal_labels',in_germinal_labels)
np.save('unlabelled_tissue_labels',unlabelled_tissue_labels)
# np.save('in_out_skew_labels',in_out_skew_labels)

In [9]:
def findlabels(a,b,c,d):
    intersect = list(set(a) & set(b)& set(c) & set(d))
    return intersect 

In [7]:
python_data_path= r"Y:\coskun-lab\Shambavi\Paper\Data\Main Data\New Data\Python Data"
results_store_path = r"Y:\coskun-lab\Shambavi\Paper\Results"
os.chdir(python_data_path)

# Functions Defined to obtain Pixel values, coordinates of Datasets

In [10]:
def pixel_values_zero(n_slices,path,onlyfiles):
    n_slices = n_slices #number of slices here
    size = 256
    pixel_value = []
    px = np.empty([len(onlyfiles),256*256*n_slices])
    py = np.empty([len(onlyfiles),256*256*n_slices])
    pz = np.empty([len(onlyfiles),256*256*n_slices])
    pv = np.empty([len(onlyfiles),256*256*n_slices])
    pixel_value = []
    x_value = [] 
    y_value = [] 
    z_value = [] 
    
    # for i in range(len(onlyfiles)):
    yay = [] 
    m = 256
    n = 256
    for i in range(len(onlyfiles)):
        with open(os.path.join(path,onlyfiles[i])) as infile:
            pixel_value = []
            x_value = []
            y_value = [] 
            z_value = []
            for _ in range(10):
                    next(infile)
            for line in infile:
                    #images_x.append(line.split()[0])
                    pixel_value.append(line.split()[3])
                    
                        
                    
                    x_value.append(int(line.split()[0]))
                    y_value.append(int(line.split()[1]))
                    z_value.append(int(line.split()[2]))
            
            pv[i,:] = pixel_value    
            px[i,:] = x_value
            py[i,:] = y_value
            pz[i,:] = z_value
        print(i)
    return pv,px,py,pz

In [11]:
def pixel_values_nonzero(n_slices,path,onlyfiles):
    n_slices = n_slices #number of slices here
    pixel_value = []
    px_count = []
    py_count = []
    pz_count = []
    pv_count = []
    x_value = [] 
    y_value = [] 
    z_value = [] 
    size = 256
    # for i in range(len(onlyfiles)):
    yay = [] 
    m = 256
    n = 256
    for i in range(len(onlyfiles)):
        with open(os.path.join(path,onlyfiles[i])) as infile:
            pixel_value = []
            x_value = []
            y_value = [] 
            z_value = []
            for _ in range(10):
                    next(infile)

            for line in infile:
                    if float(line.split()[3]) == 0:
                        continue
                    pixel_value.append(float(line.split()[3]))
                    x_value.append(int(line.split()[0]))
                    y_value.append(int(line.split()[1]))
                    z_value.append(int(line.split()[2]))

            pv_count.append(pixel_value)       
            px_count.append(x_value)
            py_count.append(y_value)
            pz_count.append(z_value)
        print(i)
    return pv_count,px_count,py_count,pz_count

In [12]:
# def retrive_pixel(pv,n_slices):
#     temp = np.empty([len(pv),256*256*n_slices])
#     for i in range(len(pv)):
#         for j in range(len(pv[0])):
# #            if float(pv[i][j][-1]) == 0:
# #                 temp[i][j] = float(pv[i][j][-1][0:3])
# #                 continue
#             temp[i][j] = float(pv[i][j][0:3])*(10^float(pv[i][j][-1]))
#     return temp 
        

In [13]:
pixel_in_zero,pixel_in_zero_x,pixel_in_zero_y,pixel_in_zero_z = pixel_values_zero(152,in_germinal,in_germinal_files)#

np.save('pixel_in_zero',pixel_in_zero)
np.save('pixel_in_zero_x',pixel_in_zero_x)
np.save('pixel_in_zero_y',pixel_in_zero_y)
np.save('pixel_in_zero_z',pixel_in_zero_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [14]:
pixel_in_nonzero,pixel_in_nonzero_x,pixel_in_nonzero_y,pixel_in_nonzero_z = pixel_values_nonzero(152,in_germinal,in_germinal_files)
np.save('pixel_in_nonzero',pixel_in_nonzero)
np.save('pixel_in_nonzero_x',pixel_in_nonzero_x)
np.save('pixel_in_nonzero_y',pixel_in_nonzero_y)
np.save('pixel_in_nonzero_z',pixel_in_nonzero_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [27]:
pixel_out_zero,pixel_out_zero_x,pixel_out_zero_y,pixel_out_zero_z = pixel_values_zero(150,out_germinal,out_germinal_files) #
np.save('pixel_out_zero',pixel_out_zero)
np.save('pixel_out_zero_x',pixel_out_zero_x)
np.save('pixel_out_zero_y',pixel_out_zero_y)
np.save('pixel_out_zero_z',pixel_out_zero_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [28]:
pixel_out_nonzero,pixel_out_nonzero_x,pixel_out_nonzero_y,pixel_out_nonzero_z = pixel_values_nonzero(150,out_germinal,out_germinal_files)
np.save('pixel_out_nonzero',pixel_out_nonzero)
np.save('pixel_out_nonzero_x',pixel_out_nonzero_x)
np.save('pixel_out_nonzero_y',pixel_out_nonzero_y)
np.save('pixel_out_nonzero_z',pixel_out_nonzero_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [22]:
pixel_in_out_zero,pixel_in_out_zero_x,pixel_in_out_zero_y,pixel_in_out_zero_z = pixel_values_zero(157,in_out_germinal,in_out_germinal_files)#
np.save('pixel_in_out_zero',pixel_in_out_zero)
np.save('pixel_in_out_zero_x',pixel_in_out_zero_x)
np.save('pixel_in_out_zero_y',pixel_in_out_zero_y)
np.save('pixel_in_out_zero_z',pixel_in_out_zero_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [23]:
pixel_in_out_nonzero,pixel_in_out_nonzero_x,pixel_in_out_nonzero_y,pixel_in_out_nonzero_z = pixel_values_nonzero(157,in_out_germinal,in_out_germinal_files)
np.save('pixel_in_out_nonzero',pixel_in_out_nonzero)
np.save('pixel_in_out_nonzero_x',pixel_in_out_nonzero_x)
np.save('pixel_in_out_nonzero_y',pixel_in_out_nonzero_y)
np.save('pixel_in_out_nonzero_z',pixel_in_out_nonzero_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [35]:
pixel_unlabelled_zero,pixel_unlabelled_zero_x,pixel_unlabelled_zero_y,pixel_unlabelled_zero_z = pixel_values_zero(138,unlabelled_tissue,unlabelled_tissue_files)
np.save('pixel_unlabelled_zero',pixel_unlabelled_zero) 
np.save('pixel_unlabelled_zero_x',pixel_unlabelled_zero_x)
np.save('pixel_unlabelled_zero_y',pixel_unlabelled_zero_y)
np.save('pixel_unlabelled_zero_z',pixel_unlabelled_zero_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [36]:
pixel_unlabelled_nonzero,pixel_unlabelled_nonzero_x,pixel_unlabelled_nonzero_y,pixel_unlabelled_nonzero_z = pixel_values_nonzero(138,unlabelled_tissue,unlabelled_tissue_files)
np.save('pixel_unlabelled_nonzero',pixel_unlabelled_nonzero)
np.save('pixel_unlabelled_nonzero_x',pixel_unlabelled_nonzero_x)
np.save('pixel_unlabelled_nonzero_y',pixel_unlabelled_nonzero_y)
np.save('pixel_unlabelled_nonzero_z',pixel_unlabelled_nonzero_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [18]:
del pixel_unlabelled_zero
del pixel_unlabelled_zero_x
del pixel_unlabelled_zero_y
del pixel_unlabelled_zero_z

In [ ]:
del pixel_in_nonzero
del pixel_in_nonzero_x
del pixel_in_nonzero_y
del pixel_in_nonzero_z

In [19]:
del pixel_in_zero
del pixel_in_zero_x
del pixel_in_zero_y
del pixel_in_zero_z

In [20]:
del pixel_in_nonzero
del pixel_in_nonzero_x
del pixel_in_nonzero_y
del pixel_in_nonzero_z

In [33]:
del pixel_out_zero
del pixel_out_zero_x
del pixel_out_zero_y
del pixel_out_zero_z

In [34]:
del pixel_out_nonzero
del pixel_out_nonzero_x
del pixel_out_nonzero_y
del pixel_out_nonzero_z

# Resize the size of numpy files


In [ ]:
pixel_in = np.load('./data/pixel_in_out_zero.npy',allow_pickle=True)

In [ ]:
pixel_in = pixel_in.astype(np.uint8)
np.save('pixel_in_out_zero_int',pixel_in) 